# Correlation of time based resolutions (OHLCV) with daily trend

Analysis of how different timeframes (12s to 1h) relate to daily market movements. Uses adaptive EMAs that scale with timeframe length (e.g., 8/21 base periods adjusted by √(timeframe/60)) to evaluate:

- Trend Alignment: Daily direction correlation
- Early Trend Detection: Morning pattern alignment 
- Signal Consistency: Stability of direction
- Volatility Capture: Range ratio vs daily movement

Features market hours filtering (9:30-16:00), visualizations and weighted scoring to understand relationships between different time resolutions and daily trend.

Input required: 1-second OHLCV data.

In [1]:
import vectorbtpro as vbt
import ttools as tts
#from lightweight_charts import chart, Panel, PlotDFAccessor, PlotSRAccessor
#import talib
import ttools as tts
from ttools.config import DATA_DIR
from ttools.utils import zoneNY, AggType
from ttools.loaders import load_data
from numba import jit
import pandas as pd
import numpy as np
from datetime import datetime
vbt.settings.plotting.auto_rangebreaks = True
vbt.settings.set_theme("dark")
vbt.settings.plotting["use_resampler"] = True

from vectorbtpro.utils.config import merge_dicts, Config, HybridConfig
from vectorbtpro import _typing as tp
from vectorbtpro.generic import nb as generic_nb

_feature_config: tp.ClassVar[Config] = HybridConfig(
    {
        "buyvolume": dict(
            resample_func=lambda self, obj, resampler: obj.vbt.resample_apply(
                resampler,
                generic_nb.sum_reduce_nb,
            )
        ),
        "sellvolume": dict(
            resample_func=lambda self, obj, resampler: obj.vbt.resample_apply(
                resampler,
                generic_nb.sum_reduce_nb,
            )
        ),
        "trades": dict(
            resample_func=lambda self, obj, resampler: obj.vbt.resample_apply(
                resampler,
                generic_nb.sum_reduce_nb,
            )
        )
    }
)



TTOOLS: Loaded env variables from file /Users/davidbrazda/Documents/Development/python/.env


In [2]:
#This is how to call LOAD function
symbol = ["BAC"]
#datetime in zoneNY 
day_start = datetime(2024, 1, 1, 9, 30, 0)
day_stop = datetime(2024, 10, 20, 16, 0, 0)
day_start = zoneNY.localize(day_start)
day_stop = zoneNY.localize(day_stop)

#requested AGG
resolution = 1 #12s bars
agg_type = AggType.OHLCV #other types AggType.OHLCV_VOL, AggType.OHLCV_DOL, AggType.OHLCV_RENKO
exclude_conditions = ['C','O','4','B','7','V','P','W','U','Z','F','9','M','6'] #None to defaults
minsize = 100 #min trade size to include
main_session_only = True
force_remote = False

bac_data = load_data(symbol = symbol,
                     agg_type = agg_type,
                     resolution = resolution,
                     start_date = day_start,
                     end_date = day_stop,
                     #exclude_conditions = None,
                     minsize = minsize,
                     main_session_only = main_session_only,
                     force_remote = force_remote,
                     return_vbt = True, #returns vbt object
                     verbose = True
                     )

matched agg files 0
BAC Contains 202  market days
BAC All 193 split files loaded in 27.000663995742798 seconds
Trimming 2024-01-01 09:30:00-05:00 2024-10-20 16:00:00-04:00
excluding ['4', '6', '7', '9', 'B', 'C', 'F', 'M', 'O', 'P', 'U', 'V', 'W', 'Z']
exclude done
minsize 100
minsize done
BAC filtered


BAC Remote fetching:  44%|████▍     | 4/9 [00:00<00:00, 38.18it/s]

Fetching from remote.Fetching from remote.

Fetching from remote.
Fetching from remote.


BAC Remote fetching: 100%|██████████| 9/9 [00:00<00:00, 65.28it/s]


Fetching from remote.
Fetching from remote.
Fetching from remote.
Fetching from remote.
Fetching from remote.


...

Remote fetched completed whole day 2024-01-11
Exact UTC range fetched: 2024-01-11 05:00:00+00:00 - 2024-01-12 04:59:59.999999+00:00


BAC Receiving trades:  89%|████████▉ | 8/9 [00:52<00:03,  3.52s/it]

Saved to CACHE /Users/davidbrazda/Library/Application Support/v2realbot/tradecache/BAC-2024-01-11.parquet
Trimming 2024-01-11 00:00:00-05:00 2024-01-11 23:59:59.999999-05:00
excluding ['4', '6', '7', '9', 'B', 'C', 'F', 'M', 'O', 'P', 'U', 'V', 'W', 'Z']
exclude done
minsize 100
minsize done
Remote fetched completed whole day 2024-01-12
Exact UTC range fetched: 2024-01-12 05:00:00+00:00 - 2024-01-13 04:59:59.999999+00:00
Saved to CACHE /Users/davidbrazda/Library/Application Support/v2realbot/tradecache/BAC-2024-01-12.parquet
Trimming 2024-01-12 00:00:00-05:00 2024-01-12 23:59:59.999999-05:00
excluding ['4', '6', '7', '9', 'B', 'C', 'F', 'M', 'O', 'P', 'U', 'V', 'W', 'Z']
exclude done
minsize 100
minsize done


BAC Receiving trades: 100%|██████████| 9/9 [01:19<00:00,  8.79s/it]


BAC Saved to agg_cache /Users/davidbrazda/Library/Application Support/v2realbot/aggcache/BAC-AggType.OHLCV-1-2024-01-01T09-30-00-2024-10-20T16-00-00-4679BCFMOPUVWZ-100-True.parquet


In [36]:
bac_data._feature_config = _feature_config

res_data = bac_data[['open', 'high', 'low', 'close', 'volume']].resample("1D")
#res_data = res_data.transform(lambda df: df.between_time('09:30', '16:00').dropna())


res_data.data["BAC"]


,open,high,low,close,volume
time,,,,,
2024-10-01 00:00:00-04:00,39.52,39.5360,38.9400,39.1000,14717405.0
2024-10-02 00:00:00-04:00,39.20,39.5400,39.0300,39.2300,14571696.0
2024-10-03 00:00:00-04:00,39.07,39.2899,38.7250,39.2700,13968494.0
2024-10-04 00:00:00-04:00,39.26,40.2500,39.2600,40.0505,19414200.0
2024-10-05 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN
2024-10-06 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN
2024-10-07 00:00:00-04:00,40.06,40.4350,39.7101,39.9300,14750102.0
2024-10-08 00:00:00-04:00,39.81,40.1500,39.8050,39.9015,15025260.0
2024-10-09 00:00:00-04:00,39.99,40.3850,39.7200,40.0600,17112678.0


In [3]:
import pandas as pd
import numpy as np
import plotly.express as px

def get_trend(df, timeframe_seconds):
    """Calculate trend using timeframe-scaled EMAs"""
    # Scale EMA periods based on timeframe
    scale_factor = timeframe_seconds / 60
    
    # Round to nearest integer but ensure minimum periods
    fast_period = max(3, round(8 * np.sqrt(scale_factor)))
    slow_period = max(6, round(21 * np.sqrt(scale_factor)))
    
    # Calculate scaled EMAs
    df['ema_fast'] = df['close'].ewm(span=fast_period).mean()
    df['ema_slow'] = df['close'].ewm(span=slow_period).mean()
    
    # Combine signals with more weight on EMAs for stability
    trend = np.sign(
        0.3 * (df['close'] - df['open']) +  # Price action (30% weight)
        0.7 * (df['ema_fast'] - df['ema_slow'])  # Trend structure (70% weight)
    )
    return trend, fast_period, slow_period

def resample_data(symbol, vbtobj, timeframe):
    """Resample data to specified timeframe"""
    res_data = vbtobj[['open', 'high', 'low', 'close', 'volume']].resample(f"{timeframe}s")
    if timeframe < 86400:
        res_data = res_data.transform(lambda df: df.between_time('09:30', '16:00').dropna())
    return res_data.data[symbol]

def analyze_trend_predictions(base_data, symbol, min_seconds=12, max_seconds=3600):
    """Analyze how well different timeframe resolutions predict daily trends."""
    
    # Generate timeframes to test
    timeframes = []
    current_tf = min_seconds
    while current_tf <= max_seconds:
        timeframes.append(current_tf)
        if current_tf < 60:
            current_tf += 12
        elif current_tf < 300:
            current_tf += 60
        else:
            current_tf += 300

    # Get daily data and calculate its trend
    daily_df = resample_data(symbol, base_data, 86400)
    daily_trend, daily_fast, daily_slow = get_trend(daily_df, 86400)
    daily_df['daily_trend'] = daily_trend
    
    results = []
    
    for tf in timeframes:
        # Resample data to current timeframe
        tf_df = resample_data(symbol, base_data, tf)
        
        if tf_df.empty:
            continue
            
        # Calculate trends with scaled EMAs
        tf_trend, fast_period, slow_period = get_trend(tf_df, tf)
        tf_df['tf_trend'] = tf_trend
        
        # Calculate metrics per day
        daily_metrics = []
        
        for date, group in tf_df.groupby(tf_df.index.date):
            # Get daily trend for this date
            try:
                daily_row = daily_df[daily_df.index.date == date].iloc[0]
                daily_trend = daily_row['daily_trend']
                
                # Ensure we have enough data for this day
                if len(group) < 4:  # Minimum 4 bars per day
                    continue
                
                # Split day into periods
                day_quarters = np.array_split(group, 4)
                
                metrics = {
                    'timeframe': tf,
                    'timeframe_readable': format_seconds(tf),
                    'fast_period': fast_period,
                    'slow_period': slow_period,
                    
                    # Trend alignment (how well does this timeframe predict daily trend)
                    'trend_alignment': np.mean(tf_trend == daily_trend),
                    
                    # Early prediction (first quarter of day)
                    'early_trend_alignment': np.mean(day_quarters[0]['tf_trend'] == daily_trend),
                    
                    # Trend consistency (how stable are the signals)
                    'trend_consistency': 1 - (np.diff(tf_trend) != 0).mean(),
                    
                    # Range ratio (normalized to daily range)
                    'avg_range_ratio': (
                        (group['high'].max() - group['low'].min()) / 
                        (daily_row['high'] - daily_row['low'])
                    )
                }
                
                daily_metrics.append(metrics)
                
            except (IndexError, KeyError):
                continue
        
        if not daily_metrics:
            continue
            
        # Average metrics across days
        metrics_df = pd.DataFrame(daily_metrics).dropna()
        avg_metrics = metrics_df.select_dtypes(include=[np.number]).mean()
        
        # Add back non-numeric columns we want to keep
        avg_metrics['timeframe'] = metrics_df['timeframe'].iloc[0]
        avg_metrics['timeframe_readable'] = metrics_df['timeframe_readable'].iloc[0]
        
        results.append(avg_metrics)
    
    return pd.DataFrame(results)

def format_seconds(seconds):
    """Convert seconds to readable format"""
    if seconds < 60:
        return f"{seconds}s"
    elif seconds < 3600:
        minutes = seconds // 60
        remaining_seconds = seconds % 60
        if remaining_seconds == 0:
            return f"{minutes}m"
        return f"{minutes}m{remaining_seconds}s"
    else:
        hours = seconds // 3600
        remaining_minutes = (seconds % 3600) // 60
        if remaining_minutes == 0:
            return f"{hours}h"
        return f"{hours}h{remaining_minutes}m"

def plot_metrics_heatmap(results_df):
    """Create a heatmap visualization of the metrics"""
    metrics = ['trend_alignment', 'early_trend_alignment', 
               'trend_consistency', 'avg_range_ratio']
    
    metric_labels = {
        'trend_alignment': 'Overall Trend Alignment',
        'early_trend_alignment': 'Early Day Trend Alignment',
        'trend_consistency': 'Trend Consistency',
        'avg_range_ratio': 'Avg Range Ratio'
    }
    
    # Prepare data for plotting
    plot_df = results_df.set_index('timeframe_readable')[metrics]
    
    # Create heatmap
    fig = px.imshow(
        plot_df.T,
        labels=dict(x='Timeframe', y='Metric', color='Score'),
        title='Timeframe Resolution Analysis with Scaled EMAs',
        color_continuous_scale='RdYlBu',
        aspect='auto'
    )
    
    # Update layout
    fig.update_layout(
        xaxis_title='Timeframe Resolution (with EMA periods)',
        yaxis_title='Metric',
        xaxis={'tickangle': 45},
        yaxis={'ticktext': list(metric_labels.values()),
               'tickvals': list(range(len(metrics)))}
    )
    
    fig.show()
    return

def create_visualizations(results_df):
    """Create multiple visualizations for timeframe analysis"""
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    # 1. Main metrics line plot
    fig1 = go.Figure()
    
    metrics = ['trend_alignment', 'early_trend_alignment', 'trend_consistency', 'avg_range_ratio']
    colors = ['rgb(31, 119, 180)', 'rgb(255, 127, 14)', 
              'rgb(44, 160, 44)', 'rgb(214, 39, 40)']
    
    for metric, color in zip(metrics, colors):
        fig1.add_trace(go.Scatter(
            x=results_df['timeframe'],
            y=results_df[metric],
            name=metric.replace('_', ' ').title(),
            line=dict(color=color),
            mode='lines+markers'
        ))
    
    fig1.update_layout(
        title='Metrics vs Timeframe Resolution',
        xaxis_title='Timeframe (seconds)',
        yaxis_title='Score',
        hovermode='x unified',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )
    
    # 2. EMA Periods vs Performance
    fig2 = make_subplots(rows=1, cols=2, 
                        subplot_titles=('Fast EMA Period vs Overall Score', 
                                      'Slow EMA Period vs Overall Score'))
    
    fig2.add_trace(
        go.Scatter(x=results_df['fast_period'], 
                  y=results_df['overall_score'],
                  mode='markers',
                  marker=dict(
                      size=10,
                      color=results_df['timeframe'],
                      colorscale='Viridis',
                      showscale=True,
                      colorbar=dict(title='Timeframe (s)')
                  ),
                  name='Fast EMA'),
        row=1, col=1
    )
    
    fig2.add_trace(
        go.Scatter(x=results_df['slow_period'], 
                  y=results_df['overall_score'],
                  mode='markers',
                  marker=dict(
                      size=10,
                      color=results_df['timeframe'],
                      colorscale='Viridis',
                      showscale=False
                  ),
                  name='Slow EMA'),
        row=1, col=2
    )
    
    fig2.update_layout(
        title='EMA Periods Impact on Performance',
        height=500,
        showlegend=False,
        hovermode='closest'
    )
    
    # 3. Performance matrix
    fig3 = make_subplots(rows=1, cols=2,
                        subplot_titles=('Trend Prediction Performance',
                                      'Signal Quality Metrics'))
    
    # Trend prediction performance
    fig3.add_trace(
        go.Bar(x=results_df['timeframe_readable'],
               y=results_df['trend_alignment'],
               name='Overall Trend',
               marker_color='rgb(31, 119, 180)'),
        row=1, col=1
    )
    
    fig3.add_trace(
        go.Bar(x=results_df['timeframe_readable'],
               y=results_df['early_trend_alignment'],
               name='Early Trend',
               marker_color='rgb(255, 127, 14)'),
        row=1, col=1
    )
    
    # Signal quality metrics
    fig3.add_trace(
        go.Bar(x=results_df['timeframe_readable'],
               y=results_df['trend_consistency'],
               name='Consistency',
               marker_color='rgb(44, 160, 44)'),
        row=1, col=2
    )
    
    fig3.add_trace(
        go.Bar(x=results_df['timeframe_readable'],
               y=results_df['avg_range_ratio'],
               name='Range Ratio',
               marker_color='rgb(214, 39, 40)'),
        row=1, col=2
    )
    
    fig3.update_layout(
        title='Detailed Performance Analysis by Timeframe',
        height=500,
        showlegend=True,
        legend=dict(orientation="h",
                   yanchor="bottom",
                   y=1.02,
                   xanchor="right",
                   x=1)
    )
    
    fig3.update_xaxes(tickangle=45)
    
    # 4. Correlation analysis
    corr_matrix = results_df[['trend_alignment', 'early_trend_alignment', 
                             'trend_consistency', 'avg_range_ratio', 
                             'overall_score']].corr()
    
    fig4 = go.Figure(data=go.Heatmap(
        z=corr_matrix,
        x=corr_matrix.columns,
        y=corr_matrix.index,
        colorscale='RdBu',
        zmin=-1,
        zmax=1,
        text=np.round(corr_matrix, 2),
        texttemplate='%{text}',
        textfont={"size": 10},
        hoverongaps=False
    ))
    
    fig4.update_layout(
        title='Correlation Matrix of Metrics',
        height=500,
        width=700
    )

    fig1.show()
    fig2.show()
    fig3.show()
    fig4.show()
    return

def main(base_data, symbol):
    """Main function to run the analysis"""
    
    # Run analysis
    results_df = analyze_trend_predictions(base_data, symbol, min_seconds=12, max_seconds=3600) #tested resolutions

    # Create visualizations
    plot_metrics_heatmap(results_df)

    # Calculate overall score with weighted metrics
    weights = {
        'trend_alignment': 0.4,
        'early_trend_alignment': 0.3,
        'trend_consistency': 0.2,
        'avg_range_ratio': 0.1
    }

    results_df['overall_score'] = sum(
        results_df[metric] * weight 
        for metric, weight in weights.items()
    )

    create_visualizations(results_df)

    # Display top 5 timeframes
    print("\nTop 5 timeframes by overall weighted score:")
    top_5 = results_df.nlargest(5, 'overall_score')
    for _, row in top_5.iterrows():
        print(f"{row['timeframe_readable']} "
              f"(EMAs: {int(row['fast_period'])}/{int(row['slow_period'])})")
        print(f"- Overall Score: {row['overall_score']:.4f}")
        print(f"- Trend Alignment: {row['trend_alignment']:.4f}")
        print(f"- Early Trend Alignment: {row['early_trend_alignment']:.4f}")
        print(f"- Trend Consistency: {row['trend_consistency']:.4f}")
        print(f"- Avg Range Ratio: {row['avg_range_ratio']:.4f}\n")

In [4]:
# df = bac_data.data["BAC"]

main(bac_data[["open", "high", "low", "close","volume"]], "BAC")


# metrics:
# # Trend alignment (how well does this timeframe predict daily trend)
# 'trend_alignment': np.mean(tf_trend == daily_trend),

# # Early prediction (first quarter of day)
# 'early_trend_alignment': np.mean(day_quarters[0]['tf_trend'] == daily_trend),

# # Trend consistency (how stable are the signals)
# 'trend_consistency': 1 - (np.diff(tf_trend) != 0).mean(),

# # Range ratio (normalized to daily range)

/Users/davidbrazda/Documents/Development/python/mlroom/.venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/davidbrazda/Documents/Development/python/mlroom/.venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/davidbrazda/Documents/Development/python/mlroom/.venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/davidbrazda/Documents/Development/python/mlroom/.venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapa


Top 5 timeframes by overall weighted score:
55m (EMAs: 59/156)
- Overall Score: 0.7829
- Trend Alignment: 0.7375
- Early Trend Alignment: 0.7228
- Trend Consistency: 0.9401
- Avg Range Ratio: 0.8302

50m (EMAs: 57/148)
- Overall Score: 0.7789
- Trend Alignment: 0.7305
- Early Trend Alignment: 0.7150
- Trend Consistency: 0.9388
- Avg Range Ratio: 0.8445

1h (EMAs: 62/163)
- Overall Score: 0.7731
- Trend Alignment: 0.7369
- Early Trend Alignment: 0.7073
- Trend Consistency: 0.9361
- Avg Range Ratio: 0.7891

45m (EMAs: 54/141)
- Overall Score: 0.7693
- Trend Alignment: 0.7223
- Early Trend Alignment: 0.7090
- Trend Consistency: 0.9368
- Avg Range Ratio: 0.8035

40m (EMAs: 51/133)
- Overall Score: 0.7586
- Trend Alignment: 0.7000
- Early Trend Alignment: 0.6874
- Trend Consistency: 0.9323
- Avg Range Ratio: 0.8587



In [45]:
fig.show()